In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, StringVar, OptionMenu
import os

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg



# -----------------------------------------------------------
#                      GLOBAL VARIABLES
# -----------------------------------------------------------
MISSING_VALUES = ["NA", "N/A", "na", r"^\\s*$", "", " ", "NaN"]

#Tracking flags
data_cleaned = False  # Tracks whether data cleaning has been performed

# DataFrames for storing user inputs
df_component_codes = None
df_activity_log = None
df_user_log = None
df_cleaned_data = None

#Pivot tables
pivot_df = None # Aggregated by user, ignoring month
monthly_pivot_df = None # Aggregated by user and month

# Folder for storing JSON data
JSON_FOLDER = "json_data"
if not os.path.exists(JSON_FOLDER):
    os.makedirs(JSON_FOLDER)

#Full list of components
all_csv_components = [
    "Assignment", "Quiz", "Lecture", "Book", "Project", "Course", "System", "Study_material", "Manual", "Folder",
    "Page", "Questionnaire", "Feedback", "Attendence", "Source", "URL", "Test", "Survey"
                     ] 

# We remove System and Folder in clean_data().
# We define componenets_of_interest 
components_of_interest = [
    "Assignment", "Quiz", "Lecture", "Attendence", "Survey", "Course", "Book", "Project"
]

# Components for correlation analysis
components_for_correlation = [
    "Assignment", "Quiz", "Lecture", "Book", "Project", "Course"
]

# -----------------------------------------------------------
#                   HELPER FUNCTIONS
# -----------------------------------------------------------

def convert_csv_to_json():
    """
    Convert selected CSVs to JSON and save in JSON_FOLDER
    """
    try:
        filepaths = filedialog.askopenfilenames(
            title="Select CSV files for conversion to JSON",
            filetypes=[("CSV Files","*.csv")]
        )
        if not filepaths:
            messagebox.showinfo("Info", "No CSV files selected.")
            return

        for fp in filepaths:
            filename = os.path.basename(fp)
            base_name = os.path.splitext(filename)[0]
            df = pd.read_csv(fp)
            json_path = os.path.join(JSON_FOLDER, f"{base_name}.json")
            df.to_json(json_path, orient='records')
            print(f"Saved JSON: {json_path}")

        messagebox.showinfo("Success", f"Converted {len(filepaths)} file(s) to JSON.")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to convert CSV to JSON: {e}")

def load_data_from_json():
    """
    Load saved JSON files (component_codes.json, activity_log.json, user_log,json) and store them in the global DataFrames.
    """
    global df_component_codes, df_activity_log, df_user_log
    try:
        component_path = os.path.join(JSON_FOLDER, "component_codes.json")
        activity_path = os.path.join(JSON_FOLDER, "activity_log.json")
        user_path = os.path.join(JSON_FOLDER, "user_log.json")

        df_component_codes = pd.read_json(component_path, orient='records')
        df_activity_log = pd.read_json(activity_path, orient='records')
        df_user_log = pd.read_json(user_path, orient='records')

        messagebox.showinfo("Success", "JSON data loaded successfully.")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load JSON data: {e}")

def clean_data():
    """
    - REMOVE 'System and 'Folder
    - RENAME 'User Full Name *Anonymized' to User_ID
    - MERGE Suitable CSVs for analysing user interactions with each componenet
    - RESHAPE: Reshape the data using the pivot operation - pivot_df (semester_ and monthyl_pivot_df (monthly_
    _ COUNT: The interactions for each user with the componenet for each month
    """
    global data_cleaned, df_cleaned_data
    global pivot_df, monthly_pivot_df
    global df_user_log, df_activity_log, df_component_codes
    
    try:
        if df_component_codes is None:
            messagebox.showwarning("Warning", "Please upload Component Codes first.")
            return
        if df_activity_log is None:
            messagebox.showwarning("Warning", "Please upload Activity Log first.")
            return
        if df_user_log is None:
            messagebox.showwarning("Warning", "Please upload User Log first.")
            return
            

        # Step 1. REMOVE: Filter out 'System' and 'Folder' components, and make a copy
        df_activity_log_removed = df_activity_log[~df_activity_log['Component'].isin(['System', 'Folder'])].copy()

        # Step 2. RENAME: 'User Full Name *Anonymized' to User_ID
        df_user_log.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)
        df_activity_log_removed.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)

        # Step 3. MERGE
        merged_df = pd.merge(df_activity_log_removed, df_user_log, on="User_ID", how="left")
        merged_df = pd.merge(merged_df, df_component_codes, on="Component", how="left")

        # Convert the Date to datetime and extract Month
        merged_df["Date"] = pd.to_datetime(merged_df["Date"], errors="coerce")
        merged_df["Month"] = merged_df["Date"].dt.month

        # Step 4 & 5: Rehape and Count
        # Group by (User_id, Componenet, Mon th), and then count the interactions
        grouped = merged_df.groupby(["User_ID", "Component", "Month"]).size().reset_index(name="Interaction_Count")

        #a) pinvot_df (entire semester): sums up across all months
        global_pivot = grouped.groupby(["User_ID", "Component"])["Interaction_Count"].sum().reset_index()
        pivot_df = global_pivot.pivot_table(
            index="User_ID",
            columns="Component",
            values="Interaction_Count",
            fill_value=0,
            aggfunc="sum"
        )

        #b) monthly_pivot_df: preserve the (User_ID, Month) index
        monthly_pivot_df = grouped.pivot_table(
            index=["User_ID", "Month"],
            columns="Component",
            values="Interaction_Count",
            fill_value=0,
            aggfunc="sum"
        )

        df_cleaned_data = merged_df
        data_cleaned = True
        messagebox.showinfo("Success", "Data cleaned and pivoted successfully!")

    except Exception as e:
        messagebox.showerror("Error", f"Error cleaning data: {e}")

def compute_statistics(component):
    """
    Compute mean, median, mode across the entire semester (in pivot_df) for entire semester.
    """
    if pivot_df is None:
        raise ValueError("pivot_df is empty. Clean your data first.")
    if component not in pivot_df.columns:
        raise ValueError(f"Component '{component}' not found in pivot_df.")

    data = pivot_df[component]
    mean_val = data.mean()
    median_val = data.median()
    
    mode_series = data.mode()
    if not mode_series.empty:
        mode_val = mode_series.iloc[0]
    else:
        mode_val = None

    return {
        "mean": mean_val,
        "median": median_val,
        "mode": mode_val

    }

def show_statistics():
    """
    GUI callback: Show entire-semester stats for a selected component.
    """
    if pivot_df is None:
        messagebox.showerror("Error", "No data available. Please upload and clean first.")
        return
    comp = selected_component.get()
    try:
        stats = compute_statistics(comp)
        out = (
            f"Component: {comp}\n"
            f"Mean: {stats['mean']:.2f}\n"
            f"Median: {stats['median']:.2f}\n"
            f"Mode: {stats['mode']}"
        )
        stats_label.config(text=out)
    except Exception as e:
        messagebox.showerror("Uh-oh, error", str(e))
        
def compute_monthly_statistics(component):
    """
    Compute monthly mean, median, mode for the given component (monthly_pivot_df).
    Returns a dict keyed by month -> {mean, median, mode}.
    """
    if monthly_pivot_df is None:
        raise ValueError("The monthly_pivot_df is empty. Can you please clean your data first?")
    if component not in monthly_pivot_df.columns:
        raise ValueError(f"Uh-oh Component '{component}' is not found in monthly_pivot_df.")

    monthly_stats = {}
    # monthly_pivot_df is indexed by (User_ID, Month) => group by 'Month' level
    for month_val, data in monthly_pivot_df[component].groupby(level="Month"):
        mmean = data.mean()
        mmedian = data.median()
        mmode_series = data.mode()
        mmode = mmode_series.iloc[0] if not mmode_series.empty else None
        monthly_stats[month_val] = {"mean": mmean, "median": mmedian, "mode": mmode}

    return monthly_stats
    
def show_monthly_statistics():
    """
    GUI callback: Show monthly stats for selected component.
    """
    if not data_cleaned:
        messagebox.showwarning("Warning", "Please clean data before computing monthly stats.")
        return

    comp = selected_component.get()
    try:
        mstats = compute_monthly_statistics(comp)
        lines = []
        for month_val in sorted(mstats.keys()):
            lines.append(
                f"Month {month_val}: Mean={mstats[month_val]['mean']:.2f}, "
                f"Median={mstats[month_val]['median']:.2f}, "
                f"Mode={mstats[month_val]['mode']}"
            )
        monthly_stats_label.config(text="\n".join(lines))
    except Exception as e:
        messagebox.showerror("Uh-oh, error", str(e))

def compute_semester_statistics():
    """
    Show entire-semester stats (mean, median, mode) for each of
    the 'components_of_interest' in pivot_df.
    """
    if pivot_df is None:
        messagebox.showerror("Error", "No pivot_df found. Please clean data first.")
        return
        
    lines = []
    for comp in components_of_interest:
        if comp in pivot_df.columns:
            stats = compute_statistics(comp)
            lines.append(
                f"{comp}: Mean={stats['mean']:.2f}, "
                f"Median={stats['median']:.2f}, Mode={stats['mode']}"
            )

    if lines:
        semester_stats_label.config(text="\n".join(lines))
    else:
        semester_stats_label.config(text=" There is no valid components found in pivot_df.")


def generate_graph():
    """
    Generate a heatmap showing correlation for the components_for_correlation
    in pivot_df (entire semester).
    """
    if pivot_df is None:
        messagebox.showerror("Error", "No data to analyse. Please clean data first.")
        return

  # Filter only the components we want AND which actually exist
    existing_cols = [c for c in components_for_correlation if c in pivot_df.columns]
    if not existing_cols:
        messagebox.showerror("Error", "No data to analyse. Please clean data first.")
        return
        
  # Calculate correlation
    selected_data = pivot_df[existing_cols]
    corr_matrix = selected_data.corr()

 # Clear any existing graph from the frame first
    for widget in graph_frame.winfo_children():
        widget.destroy()

  # Create a figure for the heatmap
    fig = plt.Figure(figsize=(7, 5), dpi=100)
    ax = fig.add_subplot(111)
    sns.heatmap(corr_matrix, ax=ax, annot=True, cmap="coolwarm", square=True)
    ax.set_title("Correlation Heatmap")
    fig.tight_layout()

  # Embed the figure in the graph_frame
    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(side="top", fill="both", expand=True)


def save_prepared_data():
    """
    Save df_cleaned_data (the merged DataFrame) to CSV or JSON.
    """
    if df_cleaned_data is None or pivot_df is None:
        messagebox.showerror("Error", "No cleaned data available to save.")
        return
    try:
        file_path = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV Files", "*.csv"), ("JSON Files", "*.json")],
        )
        if not file_path:
            return

        if file_path.endswith(".csv"):
            df_cleaned_data.to_csv(file_path, index=False)
        elif file_path.endswith(".json"):
            df_cleaned_data.to_json(file_path, orient='records')

        messagebox.showinfo("Success", f"Saved prepared data to {file_path}")
    except Exception as e:
        messagebox.showerror("Error", f"Error saving data: {e}")

# -----------------------------------------------------------
#                  TKINTER WINDOW SETUP
# -----------------------------------------------------------
window = tk.Tk()
window.title("Analysis Tool")
window.geometry("900x650")
window.resizable(False, False)

# Notebook tabs
notebook = ttk.Notebook(window)
tab_upload = ttk.Frame(notebook)
tab_clean = ttk.Frame(notebook)
tab_analysis = ttk.Frame(notebook)

notebook.add(tab_upload, text="Step 1. Upload Data")
notebook.add(tab_clean, text="Step 2. Clean Data & Prepare your Data")
notebook.add(tab_analysis, text="Step 3. Analyse Data")
notebook.pack(expand=1, fill="both")

# -----------------------------------------------------------
#                      UPLOAD FUNCTIONS
# -----------------------------------------------------------

#Upload Tab 1
ttk.Label(tab_upload, text="Step 1: Upload CSV Files").pack(pady=5)


def upload_component_codes():
    global df_component_codes
    path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not path:
        return
    try:
        df_component_codes = pd.read_csv(path, keep_default_na=False, na_values=MISSING_VALUES)
        messagebox.showinfo("Success", "Component Codes uploaded successfully!")
    except Exception as ex:
        messagebox.showerror("Error", f"Error loading Component Codes: {ex}")

def upload_activity_log():
    global df_activity_log
    path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not path:
        return
    try:
        df_activity_log = pd.read_csv(path, keep_default_na=False, na_values=MISSING_VALUES)
        messagebox.showinfo("Success", "Activity Log uploaded successfully!")
    except Exception as ex:
        messagebox.showerror("Error", f"Error loading Activity Log: {ex}")


def upload_user_log():
    global df_user_log
    path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not path:
        return
    try:
        df_user_log = pd.read_csv(path, keep_default_na=False, na_values=MISSING_VALUES)
        messagebox.showinfo("Success", "User Log uploaded successfully!")
    except Exception as ex:
        messagebox.showerror("Error", f"Error loading User Log: {ex}")

# Upload Buttons
btn_upload_component = ttk.Button(tab_upload, text="Upload Component Codes", command=upload_component_codes)
btn_upload_activity = ttk.Button(tab_upload, text="Upload Activity Log", command=upload_activity_log)
btn_upload_user = ttk.Button(tab_upload, text="Upload User Log", command=upload_user_log)

btn_upload_component.pack(pady=5)
btn_upload_activity.pack(pady=5)
btn_upload_user.pack(pady=5)

# Convert CSV to JSON
ttk.Label(tab_upload, text="Convert CSV to JSON").pack(pady=10)
btn_convert_to_json = ttk.Button(tab_upload, text="Convert CSV to JSON", command=convert_csv_to_json)
btn_convert_to_json.pack(pady=5)

# Load from JSON
ttk.Label(tab_upload, text="Load from previously converted JSON").pack(pady=10)
btn_load_json = ttk.Button(tab_upload, text="Load Data from JSON", command=load_data_from_json)
btn_load_json.pack(pady=5)

# -----------------------------------------------------------
#                      CLEAN FUNCTIONS
# -----------------------------------------------------------
#Clean Tab 2
ttk.Label(tab_clean, text="Step 2: Clean & Prepare Data").pack(pady=10)
btn_clean_data = ttk.Button(tab_clean, text="Clean Data", command=clean_data)
btn_clean_data.pack(pady=10)

# -----------------------------------------------------------
#                   ANALYSIS FUNCTIONS
# -----------------------------------------------------------
#Analysis Tab 3

# Scrollable canvas in tab_analysis, then place frames inside it
analysis_canvas = tk.Canvas(tab_analysis)
analysis_scrollbar = ttk.Scrollbar(tab_analysis, orient="vertical", command=analysis_canvas.yview)
analysis_frame = ttk.Frame(analysis_canvas)

analysis_frame.bind(
    "<Configure>",
    lambda e: analysis_canvas.configure(scrollregion=analysis_canvas.bbox("all"))
)

analysis_canvas.create_window((0, 0), window=analysis_frame, anchor="nw")

analysis_canvas.configure(yscrollcommand=analysis_scrollbar.set)
analysis_canvas.pack(side="left", fill="both", expand=True)
analysis_scrollbar.pack(side="right", fill="y")

# 1) Single-Component Stats (Entire Semester)
single_stats_frame = ttk.LabelFrame(analysis_frame, text="Single-Component Stats (Entire Semester)")
single_stats_frame.pack(fill="x", pady=10, padx=10)

selected_component = tk.StringVar()
selected_component.set(components_of_interest[0])  # default

ttk.Label(single_stats_frame, text="Select Component:").pack(side="left", padx=5)
comp_menu = ttk.OptionMenu(single_stats_frame, selected_component, components_of_interest[0], *components_of_interest)
comp_menu.pack(side="left", padx=5)

stats_button = ttk.Button(single_stats_frame, text="Get Stats", command=show_statistics)
stats_button.pack(side="left", padx=5)

stats_label = tk.Label(single_stats_frame, text="", justify="left")
stats_label.pack(side="left", padx=15)


# 2) Monthly Stats
monthly_frame = ttk.LabelFrame(analysis_frame, text="Monthly Stats for Selected Component")
monthly_frame.pack(fill="x", pady=10, padx=10)

monthly_button = ttk.Button(monthly_frame, text="Get Monthly Stats", command=show_monthly_statistics)
monthly_button.pack(side="left", padx=5)

monthly_stats_label = tk.Label(monthly_frame, text="", justify="left")
monthly_stats_label.pack(side="left", padx=15)

# 3) Entire-Semester Stats (All interest components)
semester_frame = ttk.LabelFrame(analysis_frame, text="Entire-Semester Stats (All Components of Interest)")
semester_frame.pack(fill="x", pady=10, padx=10)

semester_stats_button = ttk.Button(semester_frame, text="Get Semester Stats", command=compute_semester_statistics)
semester_stats_button.pack(side="left", padx=5)

semester_stats_label = tk.Label(semester_frame, text="", justify="left")
semester_stats_label.pack(side="left", padx=15)

# 4) Correlation Heatmap
graph_frame = ttk.LabelFrame(analysis_frame, text="Correlation Heatmap")
graph_frame.pack(fill="both", pady=10, padx=10)

graph_button = ttk.Button(graph_frame, text="Generate Correlation Graph", command=generate_graph)
graph_button.pack(side="top", padx=5, pady=5)

# 5) Save Data
save_data_frame = ttk.LabelFrame(analysis_frame, text="Save Merged/Cleaned Data")
save_data_frame.pack(fill="x", pady=10, padx=10)

btn_save_data = ttk.Button(save_data_frame, text="Save Prepared Data", command=save_prepared_data)
btn_save_data.pack(side="left", padx=5)

# -----------------------------------------------------------
#               MAINLOOP
# -----------------------------------------------------------
window.mainloop()

Saved JSON: json_data/ACTIVITY_LOG.json
